In [1]:
!pip install tensorflow

In [2]:
!pip install torch
!pip install transformers
!pip install datasets>=2.11

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
pip install transformers pandas scikit-learn torch tqdm

In [4]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import torch
from transformers import XLNetTokenizer, XLNetModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from datasets import load_dataset
import pprint

In [6]:
# Load the dataset
emotions_dataset = load_dataset("dair-ai/emotion")
print(emotions_dataset)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [7]:
# Extract texts and labels
train_texts = emotions_dataset['train']['text']
train_labels = emotions_dataset['train']['label']
val_texts = emotions_dataset['validation']['text']
val_labels = emotions_dataset['validation']['label']

# Encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)



In [8]:
# Tokenize the input data using XLNetTokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased')

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

train_embeddings = []
val_embeddings = []



spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

In [9]:
# Process train texts
for text in tqdm(train_texts, desc="Processing train texts"):
    embedding = get_embeddings(text)
    train_embeddings.append(embedding)

# Process validation texts
for text in tqdm(val_texts, desc="Processing validation texts"):
    embedding = get_embeddings(text)
    val_embeddings.append(embedding)

train_embeddings = torch.cat(train_embeddings).numpy()
val_embeddings = torch.cat(val_embeddings).numpy()



Processing validation texts: 100%|██████████| 2000/2000 [07:32<00:00,  4.42it/s]


In [11]:
import warnings
warnings.filterwarnings('ignore')

# Train a classifier using the embeddings
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embeddings, train_labels)

# Predict on the validation set
val_predictions = classifier.predict(val_embeddings)

# accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f'Validation Accuracy: {accuracy}')

Validation Accuracy: 0.6365


In [12]:

from sklearn.metrics import classification_report

# Predict on the validation set
val_predictions = classifier.predict(val_embeddings)

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f'Validation Accuracy: {accuracy}')

# Get classification report
report = classification_report(val_labels, val_predictions)
print(f'Classification Report:\n{report}')


              precision    recall  f1-score   support

           0       0.62      0.71      0.66       550
           1       0.72      0.77      0.74       704
           2       0.49      0.40      0.44       178
           3       0.60      0.49      0.54       275
           4       0.55      0.50      0.53       212
           5       0.49      0.32      0.39        81

    accuracy                           0.64      2000
   macro avg       0.58      0.53      0.55      2000
weighted avg       0.63      0.64      0.63      2000

